In [1]:
from sklearn.naive_bayes import MultinomialNB # ideal für counting features wie bow oder tfidf https://towardsdatascience.com/why-how-to-use-the-naive-bayes-algorithms-in-a-regulated-industry-with-sklearn-python-code-dbd8304ab2cf
from sklearn.naive_bayes import GaussianNB # für Features in Decimal Form geeignet
from sklearn.naive_bayes import ComplementNB # ähnlich wie Multinomial, soll sich aber besser für imbalanced data eignen
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    recall_score,
    precision_score,
    classification_report,
)
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

In [2]:
def evaluate(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))

## Evaluation neue Vectorize-Funktionen (08.12.)

In [3]:
%run ../../functions/vectorize_functions.py

In [5]:
filepath_name = (('../../../data/mixed_dataset/train_cleaned.csv'))
df_cleaned = pd.read_csv(filepath_name, encoding='utf-8')

In [6]:
X_train, X_test, y_train, y_test = vectorize_w2v(df=df_cleaned, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(type(X_train))

(57332, 300)
(24572, 300)
(57332,)
(24572,)
<class 'numpy.ndarray'>


## GaussianNB

In [8]:
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [9]:
evaluate(y_test, y_pred)

Accuracy: 0.6369037929350481
F1 Score: 0.4260903126206098
Recall: 0.7498302014942269
Precision: 0.29760086261119595
       0     1
0  12338  7817
1   1105  3312


In [10]:
results_list = []

param_grid = {
    'priors' : [None, [.25,.75], [.5,.5]],
    'var_smoothing' : [1.0e-10,1e-9, 0.01, 0.1, 0.25, 0.5, 0.75, 1]
}

nb = GaussianNB()

grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for params in grid_search.cv_results_['params']:
    model = GaussianNB(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    result_dict = {
        'priors': params['priors'],
        'var_smoothing': params['var_smoothing'],
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_f1': train_f1,
        'test_accuracy': test_accuracy,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f1': test_f1
    }

    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

results_df.to_csv('eval_data/nb_grid_w2v_mixed_dataset.csv', index=False)

print(results_df)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
          priors  var_smoothing  train_accuracy  train_recall  \
0           None   1.000000e-10        0.649201      0.747967   
1           None   1.000000e-09        0.649201      0.747967   
2           None   1.000000e-02        0.646114      0.752130   
3           None   1.000000e-01        0.623666      0.782823   
4           None   2.500000e-01        0.600241      0.809256   
5           None   5.000000e-01        0.581386      0.826975   
6           None   7.500000e-01        0.576205      0.831042   
7           None   1.000000e+00        0.576606      0.827169   
8   [0.25, 0.75]   1.000000e-10        0.602979      0.802188   
9   [0.25, 0.75]   1.000000e-09        0.602979      0.802188   
10  [0.25, 0.75]   1.000000e-02        0.598619      0.807901   
11  [0.25, 0.75]   1.000000e-01        0.567257      0.841983   
12  [0.25, 0.75]   2.500000e-01        0.528850      0.874032   
13  [0.25, 0.75]   5.000000e-